In [8]:
import argparse
import torch
from torchvision import transforms

from retinanet import model
from retinanet.dataloader import fileDataset, Resizer, Normalizer
from retinanet import csv_eval

assert torch.__version__.split('.')[0] == '1'
print('CUDA available: {}'.format(torch.cuda.is_available()))

CUDA available: True


In [2]:
# import images dataset
import os
from PIL import Image
#import cv2


BATCH_SIZE = 16
IMG_SIZE = (480, 640)
NUM_EPOCHS = 10

#dataset_description_path = "C:\Users\Drake Li\Desktop\cwd\ds-stuff\data\list.txt"
dataset_path = "./data/cloth-defect-detect"
anno_path="./data/cloth-defect-detect/anno_train.json"
labels_path = "./cloth-defect-detect-classes.json"
model_path = "./cloth-defect-detect.pth"    #save to here


In [3]:
import random
import json
import os

dataset=[]

classes = {}
labelcounter=0
annolabels=None

img_fn_dict={}


for i,fn in enumerate(os.listdir(dataset_path)):
    datapath=os.path.join(dataset_path,fn)
    if(os.path.isdir(datapath)==False):
        print(datapath)
        continue
    #either defect or normal
    if(fn=="defect"):
        
        #load defect
        for j,fn_defect in enumerate(os.listdir(datapath)):
            #print(fn_defect)
            img_fullpath=os.path.join(datapath,fn_defect,fn_defect+".jpg")
            img_fn_dict[fn_defect+".jpg"]=img_fullpath
            #print(img_fullpath)
            
            continue
#             img_bytes = Image.open(img_fullpath)
#             img = torchvision.transforms.functional.resize(img=img_bytes,size=IMG_SIZE)#resize img
#             img=(np.array(img)/255.0).astype(np.float32)
#             print(img.shape)
print(len(img_fn_dict))
for item in img_fn_dict:
    print(item)
    print(img_fn_dict[item])
    break
    
dataset_val =fileDataset(dataset_path,anno_path,img_fn_dict,IMG_SIZE=IMG_SIZE,transform=transforms.Compose([Normalizer(), Resizer()]))

./data/cloth-defect-detect\anno_train.json
2047
0907A1_081eed8e1d64b06a1201909071528592.jpg
./data/cloth-defect-detect\defect\0907A1_081eed8e1d64b06a1201909071528592\0907A1_081eed8e1d64b06a1201909071528592.jpg
files:2047
anno len:16457
{'缝头': 0, '缝头印': 1, '沾污': 2, '漏印': 3, '虫粘': 4, '花毛': 5, '水印': 6, '色差': 7, '褶子': 8, '破洞': 9, '织疵': 10, '蜡斑': 11, '错花': 12, '其他': 13, '网折': 14}
{0: '缝头', 1: '缝头印', 2: '沾污', 3: '漏印', 4: '虫粘', 5: '花毛', 6: '水印', 7: '色差', 8: '褶子', 9: '破洞', 10: '织疵', 11: '蜡斑', 12: '错花', 13: '其他', 14: '网折'}
[{'x1': 1, 'x2': 4087, 'y1': 1073, 'y2': 1198, 'class': '缝头'}]
[{'x1': 4, 'x2': 4085, 'y1': 760, 'y2': 881, 'class': '缝头印'}]
[{'x1': 2111, 'x2': 2182, 'y1': 1561, 'y2': 1596, 'class': '沾污'}]
[{'x1': 0, 'x2': 390, 'y1': 0, 'y2': 1691, 'class': '漏印'}]
[{'x1': 0, 'x2': 4096, 'y1': 1116, 'y2': 1205, 'class': '缝头'}]


In [4]:
retinanet=torch.load(model_path)

In [5]:
retinanet = retinanet.cuda()
retinanet.training = False
retinanet.eval()
retinanet.module.freeze_bn()

In [6]:
print(dataset_val)

In [7]:
return_vals=csv_eval.evaluate(dataset_val, retinanet,iou_threshold=0.5)
print(return_vals)

2047/2047
mAP:
缝头: 0.0


IndexError: index -1 is out of bounds for axis 0 with size 0